In [ ]:
#Importando bibliotecas
import torch
import pandas as pd
import numpy as np

# Importando dataset utils
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import importlib
if importlib.util.find_spec('ipywidgets') is not None:
    from tqdm.auto import tqdm
else:
    from tqdm import tqdm

import networkx as nx

from typing import Union

In [ ]:
dataframe = pd.read_csv('/content/data.csv', sep=';')

dataframe = dataframe.fillna(-1)
dataframe['Anos educacao formal'] = dataframe['Anos educacao formal'].replace(-5, -1)
dataframe['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
df_suic = dataframe.copy()
df_suic = df_suic.astype(float)
df_suic['Chave'] = df_suic.index
df_suic.shape

(3953, 69)

In [ ]:
notears_selected = [
    "Suic_familia",
    "Drog_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Energia",
    "Suicidio",
    "Ansiedade"]

# Lendo dados

In [ ]:
 df_novo = pd.DataFrame(columns = ["Suic_familia",
    "Drog_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Energia",
    "Suicidio",
    "Ansiedade"])

# Lendo fatores do novo indivíduo

In [ ]:
Suic_familia = int(input("Suicidio na família: Digite 1 para Sim e 0 para não.") )
Drog_familia = int(input("Drogas na família: Digite 1 para Sim e 0 para não.") )
Capaz_tomar_decisoes = int(input("Capaz de tomar decisões importantes: Digite um número de 1 a 5, sendo 1 pouca capacidade e 5 muita capacidade.") )
Estudante = int(input("Estudante: Digite 1 para Sim e 0 para não.") )
Hipocondriase = int(input("Hipocondriase: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.") )
Sentimento_culpa = int(input("Sentimentos de culpa: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.") )
Trabalho_interesses = int(input("Trabalho e interesses: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.") )
Energia = int(input("Energia: Digite um número de 0 a 2, sendo 0 não e 2 nível elevado.") )
Suicidio = int(input("Suicidio: Digite um número de 0 a 4, sendo 0 não possui ideação e 4 já efetuou alguma tentativa.") )
Ansiedade = int(input("Ansiedade: Digite um número de 0 a 4, sendo 1 pouca anciedade e 4 muita anciedade.") )

Suicidio na família: Digite 1 para Sim e 0 para não.0
Drogas na família: Digite 1 para Sim e 0 para não.1
Capaz de tomar decisões importantes: Digite um número de 1 a 5, sendo 1 pouca capacidade e 5 muita capacidade.3
Estudante: Digite 1 para Sim e 0 para não.0
Hipocondriase: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.2
Sentimentos de culpa: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.3
Trabalho e interesses: Digite um número de 0 a 4, sendo 0 não e 4 nível elevado.2
Energia: Digite um número de 0 a 2, sendo 0 não e 2 nível elevado.1
Suicidio: Digite um número de 0 a 4, sendo 0 não possui ideação e 4 já efetuou alguma tentativa.3
Ansiedade: Digite um número de 0 a 4, sendo 1 pouca anciedade e 4 muita anciedade.2


In [ ]:
df_novo=df_novo.append({'Suic_familia' : float(Suic_familia), 'Drog_familia' : float(Drog_familia), 'Capaz de tomar decisões importantes' : float(Capaz_tomar_decisoes), 'Estudante' : float(Estudante),
                        'Hipocondriase' : float(Hipocondriase), 'Sentimentos_culpa' : float(Sentimento_culpa), 'Trabalho e interesses' : float(Trabalho_interesses), 'Energia' : float(Energia),
                        'Suicidio' : float(Suicidio), 'Ansiedade': float(Ansiedade)}, ignore_index = True)

<ipython-input-52-a47f35c673b8>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_novo=df_novo.append({'Suic_familia' : float(Suic_familia), 'Drog_familia' : float(Drog_familia), 'Capaz de tomar decisões importantes' : float(Capaz_tomar_decisoes), 'Estudante' : float(Estudante),


In [ ]:
df_novo['Suic_familia'] = df_novo['Suic_familia'].astype(float)
df_novo['Drog_familia'] = df_novo['Drog_familia'].astype(float)
df_novo['Capaz de tomar decisões importantes'] = df_novo['Capaz de tomar decisões importantes'].astype(float)
df_novo['Estudante'] = df_novo['Estudante'].astype(float)
df_novo['Hipocondriase'] = df_novo['Hipocondriase'].astype(float)
df_novo['Sentimentos_culpa'] = df_novo['Sentimentos_culpa'].astype(float)
df_novo['Trabalho e interesses'] = df_novo['Trabalho e interesses'].astype(float)
df_novo['Energia'] = df_novo['Energia'].astype(float)
df_novo['Suicidio'] = df_novo['Suicidio'].astype(float)
df_novo['Ansiedade'] = df_novo['Ansiedade'].astype(float)

# Criando conjunto de dados causal

In [ ]:
# Conjunto de dados de X
class CausalDataset(Dataset):
    def __init__(self, X, target:Union[list, int]):
        self.X = torch.tensor(X, dtype=torch.double)
        self.target = target
        if isinstance(target, int):
            self.target = [target]


        # X é toda variável, exceto a lista de alvos
        dims = list(range(self.X.shape[1]))
        self.x = self.X[:, [i for i in dims if i not in self.target]]
        self.y = self.X[:, self.target]

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx].double(), self.y[idx].double()

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, dim_list:list, add_dropout:bool=False):
        super(MLP, self).__init__()
        torch.manual_seed(3)
        self.layers = torch.nn.ModuleList()
        for i in range(len(dim_list) - 2):
            self.layers.append(torch.nn.Linear(dim_list[i], dim_list[i+1]))
            self.layers.append(torch.nn.ReLU())
            if add_dropout:
                self.layers.append(torch.nn.Dropout(0.3))

        self.layers.append(torch.nn.Linear(dim_list[-2], dim_list[-1]))
        self.double()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Pipeline de treino do modelo

In [ ]:
def trainPipeline(dataset, model, batch_size=64, train_split=0.8, verbose=True, max_epochs=100, patience=10, lr=0.001, weight_decay=0.0, loss_fn=nn.MSELoss()):
    train_size = int(train_split * len(dataset))
    test_size = len(dataset) - train_size
    torch.manual_seed(3)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    criterion = loss_fn
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    patience = patience

    train_losses = []
    test_losses = []
    for epoch in tqdm(range(max_epochs)):
        train_loss = 0
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        test_loss = 0
        model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
        test_loss /= len(test_loader)
        test_losses.append(test_loss)

        if verbose:
            print(f"Epoch {epoch} - Train loss: {train_loss} - Test loss: {test_loss}")

        if epoch > 10:
            all_patience = test_losses[-patience:]
            test_loss = test_losses[-1]
            if all([test_loss > x for x in all_patience]):
                print("Early stopping")
                break

    # teste do modelo no conjunto de teste e cálculo da precisão
    model.eval()
    with torch.no_grad():
        y_pred_test = []
        y_true_test = []
        y_pred_train = []
        y_true_train = []

        for data, target in train_loader:
            output = model(data)
            y_pred_train.append(output)
            y_true_train.append(target)
        for data, target in test_loader:
            output = model(data)
            y_pred_test.append(output)
            y_true_test.append(target)

        y_pred_test = torch.cat(y_pred_test, dim=0)
        y_true_test = torch.cat(y_true_test, dim=0)
        y_pred_train = torch.cat(y_pred_train, dim=0)
        y_true_train = torch.cat(y_true_train, dim=0)

        train_error = torch.abs(y_pred_train - y_true_train)
        test_error = torch.abs(y_pred_test - y_true_test)

        # calculando acurácia para cada alvo
        test_accuracy = []
        train_accuracy = []
        for i in range(len(y_true_test[0])):
            test_accuracy.append(torch.sum(test_error[:, i] < 0.5) / len(test_error))
            train_accuracy.append(torch.sum(train_error[:, i] < 0.5) / len(train_error))

        print(f"Final Train accuracy: {train_accuracy}")
        print(f"Final Test accuracy: {test_accuracy}")

    metrics = {
        "train_loss": train_losses,
        "test_loss": test_losses,
        "train_error": train_error,
        "test_error": test_error,
        "test_accuracy": test_accuracy
    }

    return model, metrics



## Primeiro treinamos um modelo para cada alvo e obtemos o valor dos erros (individualidade)


## Agora adicionamos os erros ao conjunto de dados e treinamos o modelo contrafactual

In [ ]:
# Criando o modelo counterfactual

class CounterfactualModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CounterfactualModel, self).__init__()
        self.individuality_model = MLP([input_size, hidden_size, hidden_size, hidden_size, 1], add_dropout=True)
        self.counterfactual_model = MLP([input_size+1, hidden_size, hidden_size, hidden_size, output_size], add_dropout=True)

    def individuality(self, x):
        return self.individuality_model(x)

    def counterfactual(self, x, i):
        x = torch.cat([x, i], dim=1)
        return self.counterfactual_model(x)

    def forward(self, x):
        i = self.individuality(x)
        y = self.counterfactual(x, i)
        return y

# Agora vamos refazer tudo, mas com os fatores selecionados na inferência causal

In [ ]:
df_suic_notears = df_suic.copy()
df_suic_notears = df_suic_notears[notears_selected]
df_suic_notears=pd.concat([df_suic_notears,df_novo], ignore_index=True)

# Criando modelos para cada alvo
targets = ["Suicidio", "Ansiedade"]
targets_idx = [df_suic_notears.columns.get_loc(c) for c in targets]

# Criando base de dados
notears_suic_dataset = CausalDataset(df_suic_notears.values, targets_idx[0])
notears_anx_dataset = CausalDataset(df_suic_notears.values, targets_idx[1])
sample_x, sample_y = notears_suic_dataset[0]

# Agora prevemos os alvos com um MLP (Perceptron multicamadas)
suic_causal_model = MLP([sample_x.shape[0], 64, 64, 64, 1], add_dropout=True)
anx_causal_model = MLP([sample_x.shape[0], 64, 64, 64, 1], add_dropout=True)

suic_causal_model, suic_causal_metrics = trainPipeline(notears_suic_dataset, suic_causal_model, verbose=False, max_epochs=1000, weight_decay=1e-5)
anx_causal_model, anx_causal_metrics = trainPipeline(notears_anx_dataset, anx_causal_model, verbose=False, max_epochs=1000, weight_decay=1e-5)



  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.5191)]
Final Test accuracy: [tensor(0.3995)]


  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.5296)]
Final Test accuracy: [tensor(0.4817)]


In [ ]:
# calculando erros
suic_causal_erros = []
anx_causal_erros = []
with torch.no_grad():
    for data, y_true in notears_suic_dataset:
        y_pred = suic_causal_model(data)
        error = torch.abs(y_pred - y_true).item()
        suic_causal_erros.append(error)
    for data, y_true in notears_anx_dataset:
        y_pred = anx_causal_model(data)
        error = torch.abs(y_pred - y_true).item()
        anx_causal_erros.append(error)

suic_causal_erros = np.array(suic_causal_erros)
anx_causal_erros = np.array(anx_causal_erros)

# adicionando erros no dataframe
df_suic_notears_with_errors = df_suic_notears.copy()
df_suic_notears_with_errors["Suicidio_error"] = suic_causal_erros
df_suic_notears_with_errors["Ansiedade_error"] = anx_causal_erros

# crando dataset causal com os erros
targets_idx = [df_suic_notears_with_errors.columns.get_loc(c) for c in targets]
notears_causal_dataset_error = CausalDataset(df_suic_notears_with_errors.values, targets_idx)
sample_x, sample_y = notears_causal_dataset_error[0]

# criando modelo counterfactual
causal_cf_model = CounterfactualModel(sample_x.shape[0], 64, sample_y.shape[0])

# treinando o modelo counterfactual
causal_cf_model, causal_cf_metrics = trainPipeline(notears_causal_dataset_error, causal_cf_model, verbose=False, max_epochs=1000, weight_decay=1e-5)


  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.7970), tensor(0.8068)]
Final Test accuracy: [tensor(0.7155), tensor(0.7206)]


In [ ]:
df_suic_notears_with_errors.to_csv('/content/df_suic_notears_with_errors.csv', index=False)

In [ ]:
targets = ["Suicidio", "Ansiedade"]
df = df_suic_notears_with_errors

# caregando o modelo
counterfact = causal_cf_model
counterfact = counterfact.eval()

Suic_familia: [0.0, 1.0]
Drog_familia: [0.0, 1.0]
Capaz de tomar decisões importantes: [-1.0, 1.0, 2.0, 3.0, 4.0, 5.0]
Estudante: [-1.0, 0.0, 0.5, 1.0]
Hipocondriase: [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Sentimentos_culpa: [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Trabalho e interesses: [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Energia: [-1.0, 0.0, 1.0, 2.0]
Suicidio: [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Ansiedade: [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]


# Predição inicial

In [ ]:
def contrafactual(df):
  df_suic_notears_with_errors = df
  dataset = CausalDataset(df_suic_notears_with_errors.values, [df.columns.get_loc(c) for c in targets])

  idx = [0,3953]

  data1, y1 = dataset[idx[0]]
  data2, y2 = dataset[idx[1]]

  # prever os contrafactuais para o indivíduo
  with torch.no_grad():
      i1 = counterfact.individuality(data1.unsqueeze(0))
      i2 = counterfact.individuality(data2.unsqueeze(0))

      cf11 = counterfact.counterfactual(data1.unsqueeze(0), i1)
      cf12 = counterfact.counterfactual(data2.unsqueeze(0), i2)

      print(f"Valor previsto para ideação suicída  {cf12.squeeze(0).round().numpy().tolist()[0]}")
      print()

In [ ]:
print('Predição inicial')
contrafactual(df_suic_notears_with_errors)

Initial prediction
Counterfactual Suicide: 2.0



# Simulações contrafactuais



In [ ]:
print('Digite o valor indicado de acordo com a feature que deseja modificar')

Digite o valor indicado de acordo com a feature que deseja modificar


In [ ]:
def alter_feature(df,Variavel,valor):
  df_suic_notears_with_errors = df
  if Variavel == '1':
    df_suic_notears_with_errors['Suic_familia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Suicidio na família')
  elif Variavel == '2':
    df_suic_notears_with_errors['Drog_familia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Drogas na família')
  elif Variavel == '3':
    df_suic_notears_with_errors['Capaz de tomar decisões importantes'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Capaz de tomar decisões importantes')
  elif Variavel == '4':
    df_suic_notears_with_errors['Estudante'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Estudante')
  elif Variavel == '5':
    df_suic_notears_with_errors['Hipocondriase'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Hipocondriase')
  elif Variavel == '6':
    df_suic_notears_with_errors['Sentimentos_culpa'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Sentimentos de culpa')
  elif Variavel == '7':
    df_suic_notears_with_errors['Trabalho e interesses'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Trabalho e interesses')
  else:
    df_suic_notears_with_errors['Energia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Energia')
  return df_suic_notears_with_errors

In [ ]:
for i in range(0,8):
  df_simula = df_suic_notears_with_errors.copy()
  print('Digite o valor indicado de acordo com a feature que deseja modificar')
  Variavel = input("1 Suicidio na família, 2 Drogas na família, 3 Capaz de tomar decisões importantes, 4 Estudante, 5 Hipocondriase, 6 Sentimentos de culpa, 7 Trabalho e interesses e 8 Energia")
  Valor = input("Digite o novo valor")
  df_simula = alter_feature(df_simula, Variavel, Valor)
  contrafactual(df_simula)
  if i<7:
    simulacao = input("Deseja executar nova simulação? Digite 1 para Sim e 0 para Não")
    if simulacao=='0':
      break

Digite o valor indicado de acordo com a feature que deseja modificar
1 Suicidio na família, 2 Drogas na família, 3 Capaz de tomar decisões importantes, 4 Estudante, 5 Hipocondriase, 6 Sentimentos de culpa, 7 Trabalho e interesses e 8 Energia1
Digite o novo valor0
Suicidio na família
Counterfactual Suicide: 2.0

Deseja executar nova simulação? Digite 1 para Sim e 0 para Não
Digite o valor indicado de acordo com a feature que deseja modificar
1 Suicidio na família, 2 Drogas na família, 3 Capaz de tomar decisões importantes, 4 Estudante, 5 Hipocondriase, 6 Sentimentos de culpa, 7 Trabalho e interesses e 8 Energia1
Digite o novo valor1
Suicidio na família
Counterfactual Suicide: 3.0

Deseja executar nova simulação? Digite 1 para Sim e 0 para Não0
